In [1]:
import sys

preprocessing_path = 'C:\\Users\\joshh\\Desktop\\Uni\\Soton Year 3\\COMP3200\\fake-news-profiling\\classifier\\preprocessing'
if preprocessing_path not in sys.path:
    sys.path.insert(1, preprocessing_path)

notif_path = 'C:\\Users\\joshh\\Desktop\\Uni\\Soton Year 3\\COMP3200\\fake-news-profiling\\classifier\\notifications'
if notif_path not in sys.path:
    sys.path.insert(1, notif_path)

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow_hub import KerasLayer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from official.nlp import optimization

import ipynb.fs.full.parse_datasets as datasets
import ipynb.fs.full.preprocessing as processing
import ipynb.fs.full.bert_fake_news_classifier as bclf
from ipynb.fs.full.notif_email import send_email

# Load Dataset

In [19]:
# Load the saved dataset split
def load_data():
    return np.load("datasets/en_split_data.npy", allow_pickle=True)

(tweet_train, label_train, 
 tweet_val, label_val, 
 tweet_test, label_test) = load_data()

# Load BERT

In [39]:
# Preprocess dataset
tweet_preprocessor = processing.BertTweetFeedDataPreprocessor(
    transformers = [
        processing.tag_indicators,
        processing.replace_xml_and_html,
        processing.replace_emojis,
        processing.remove_punctuation,
        processing.replace_tags,
        processing.remove_hashtag_chars,
        processing.replace_accented_chars,
        processing.tag_numbers,
        processing.remove_stopwords,
        processing.remove_extra_spacing,
    ])
tweet_train_processed = tweet_preprocessor.transform(tweet_train)
tweet_val_processed = tweet_preprocessor.transform(tweet_val)

In [62]:
bert_encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1"
bert_hidden_layer_size = 128

bert_encoder = KerasLayer(bert_encoder_url, trainable=False)
tokenizer = bclf.BertTweetFeedTokenizer(bert_encoder, bert_hidden_layer_size)
bert_model = tf.keras.models.Model(*bclf.base_bert_model(bert_encoder, bert_hidden_layer_size))
bert_model.load_weights(
    "training/bert_feed/initial_eval/bert128-batch_size32-epochs10-lr5e-05-optimizeradamw/cp.ckpt")

In [44]:
# BERT tokenized input
X_train = tokenizer.tokenize_input(tweet_train_processed)
y_train = tokenizer.tokenize_labels(label_train)
X_validation = tokenizer.tokenize_input(tweet_val_processed)
y_validation = tokenizer.tokenize_labels(label_val)

In [74]:
# BERT predictions
with tf.device("/cpu:0"):
    X_train_bert_out = bert_model.predict(X_train)["pooled_output"]
    X_val_bert_out = bert_model.predict(X_val)["pooled_output"]

## FFNN

In [90]:
ffnn.train_step()

TypeError: train_step() missing 1 required positional argument: 'data'

In [76]:
# (128,) -> (256,) -> (128,) -> (64,) -> (1,)
ffnn = tf.keras.models.Sequential([
    Input(shape=(bert_hidden_layer_size,), dtype=tf.float32),
    Dense(bert_hidden_layer_size*2, activation="relu"),
    Dense(bert_hidden_layer_size, activation="relu"),
    Dense(bert_hidden_layer_size//2, activation="relu"),
    Dense(1, activation="sigmoid"),
])

# Optimizer
total_training_steps = epochs * len(X_train_bert_out) / batch_size
warmup_steps = int(0.1 * total_training_steps)
optimizer = optimization.create_optimizer(
    init_lr=lr,
    num_train_steps=total_training_steps,
    num_warmup_steps=warmup_steps,
    optimizer_type='adamw'
)

ffnn.compile(
    optimizer=tf.keras.optimizers.Adam(5e-5), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=tf.metrics.BinaryAccuracy(), 
)

In [86]:
with tf.device("/cpu:0"):
    train_history = ffnn.fit(
            x=X_train_bert_out,
            y=y_train,
            batch_size=32,
            epochs=40,
            validation_data=(X_val_bert_out, y_validation),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir="training/bert_clf/testing/logs/ffnn-256-128-64-1")],
    )

Epoch 1/100
  1/188 [..............................] - ETA: 0s - loss: 0.7260 - binary_accuracy: 0.5938WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.0305s). Check your callbacks.


188/188 [==============================] - 0s 2ms/step - loss: 0.6174 - binary_accuracy: 0.7544 - val_loss: 0.6169 - val_binary_accuracy: 0.7452
Epoch 2/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6165 - binary_accuracy: 0.7601 - val_loss: 0.6181 - val_binary_accuracy: 0.7371
Epoch 3/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6155 - binary_accuracy: 0.7599 - val_loss: 0.6174 - val_binary_accuracy: 0.7466
Epoch 4/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6147 - binary_accuracy: 0.7645 - val_loss: 0.6188 - val_binary_accuracy: 0.7302
Epoch 5/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6144 - binary_accuracy: 0.7679 - val_loss: 0.6189 - val_binary_accuracy: 0.7411
Epoch 6/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6136 - binary_accuracy: 0.7689 - val_loss: 0.6190 - val_binary_accuracy: 0.7398
Epoch 7/100
188/188 [==============================] - 0s 2ms/step - l

Epoch 53/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5869 - binary_accuracy: 0.8347 - val_loss: 0.6250 - val_binary_accuracy: 0.7330
Epoch 54/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5861 - binary_accuracy: 0.8380 - val_loss: 0.6251 - val_binary_accuracy: 0.7275
Epoch 55/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5856 - binary_accuracy: 0.8380 - val_loss: 0.6242 - val_binary_accuracy: 0.7302
Epoch 56/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5854 - binary_accuracy: 0.8384 - val_loss: 0.6251 - val_binary_accuracy: 0.7343
Epoch 57/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5848 - binary_accuracy: 0.8400 - val_loss: 0.6237 - val_binary_accuracy: 0.7343
Epoch 58/100
188/188 [==============================] - 0s 2ms/step - loss: 0.5845 - binary_accuracy: 0.8395 - val_loss: 0.6245 - val_binary_accuracy: 0.7302
Epoch 59/100
188/188 [==============================

In [87]:
train_history

In [ ]:
# BERT model with an LSTM output layer
def create_bert_model_lstm(encoder, input_size, **kwargs):
    bert_pooled_output = base_bert_model(encoder, input_size)['pooled_output']

    # LSTM layer output
    dense_output = LSTM(1, **kwargs)(bert_pooled_output)

    # Create the Keras model and compile
    return Model(inputs, dense_output)